## Prepare data (moved to script!)

In [ ]:
import pandas as pd

data = pd.read_csv("data/20250328_037.csv", sep=";", encoding="iso-8859-1")
data.head()

In [ ]:
# Drop empty columns
data = data.dropna(axis="columns", how="all")
data.head()

In [ ]:
# Drop columns that have all the same value
same_cols = []
metadata = {}

for colname in data.columns:
    values = data[colname].tolist()
    if len(set(values)) == 1:
        # print(f"Column {colname} is unique with value {col[0]}")
        same_cols.append(colname)
        metadata[colname] = values[0]

metadata

In [ ]:
data = data.drop(columns=same_cols)
data.head()

In [ ]:
data["WAARDEBEPALINGSMETHODE_OMSCHRIJVING"].unique()

In [ ]:
# just checking something
import numpy as np
data[data["GROEPERING_CODE"].notnull()].head()

In [ ]:
# add proper datetime column
from datetime import datetime

data["DATUMTIJD"] = pd.to_datetime(data["WAARNEMINGDATUM"] + " " + data["WAARNEMINGTIJD (MET/CET)"], format="%d-%m-%Y %H:%M:%S")

In [ ]:
# convert X and Y to proper floats
for c in ["X", "Y"]:
    data[c] = data[c].str.replace(",", ".").astype(float)


In [ ]:
# remove rows where X or Y is 0
data = data[data.X != 0]
data = data[data.Y != 0]

In [ ]:
# create locations gdf
import geopandas as gpd

key_col = "LOCATIE_CODE"
attr_cols = ["MEETPUNT_IDENTIFICATIE", "X", "Y"]

locations = data.groupby(by=key_col)[attr_cols].first()
locations = gpd.GeoDataFrame(locations, geometry=gpd.points_from_xy(locations["X"], locations["Y"], crs="EPSG:25831"))

locations.head()

In [ ]:
# convert CRS and scale to -1, 1
locations = locations.to_crs(epsg=28992)

locations["SCALED_X"] = (locations.geometry.x - 155_000) / (325_000 / 2)
locations["SCALED_Y"] = (locations.geometry.y - 463_000) / (325_000 / 2)
locations = locations.round(5)

locations.head()

In [ ]:
# select columns and export locations
locations[["MEETPUNT_IDENTIFICATIE", "SCALED_X", "SCALED_Y"]].to_csv("output/waterhoogtes_locaties.csv", sep=",", encoding="utf-8")

In [ ]:
# pivot to location per column
meas = data.pivot_table(index="DATUMTIJD", values="NUMERIEKEWAARDE", columns="LOCATIE_CODE")
meas.head()

In [ ]:
# remove RWS max values
import numpy as np
meas.replace(999_999_999.0, np.nan, inplace=True)
meas

In [ ]:
# show absolute diffs
meas.diff().abs()

In [ ]:
# remove outliers by looking at mean of absolute diff
abs_diff = meas.diff().abs()
outliers = abs_diff >= 5 * abs_diff.mean()
outliers

In [ ]:
# remove outliers
meas[outliers] = np.nan
meas

In [ ]:
# fill gaps by interpolation
meas.interpolate(axis=0, method="linear", limit_direction="both", inplace=True)
meas.astype(int)

In [ ]:
# back to long table and export
meas_long = meas.astype(int).reset_index().melt(id_vars="DATUMTIJD", value_name="NUMERIEKEWAARDE")
meas_long.to_csv("output/waterhoogtes_long.csv", sep=";", encoding="utf-8", index=False)

In [ ]:
# pivot for touch and export
meas_piv = meas_long.pivot_table(index="LOCATIE_CODE", values="NUMERIEKEWAARDE", columns="DATUMTIJD").astype(int)
meas_piv.to_csv("output/waterhoogtes_pivoted.csv", sep=",", encoding="utf-8")

In [ ]:
# unedited data request for Torben
meas_piv_selection = meas_long[meas_long["LOCATIE_CODE"].isin(["ROER", "ARNH", "NIJM", "DEVE", "ROTT"])].pivot_table(index="LOCATIE_CODE", values="NUMERIEKEWAARDE", columns="DATUMTIJD").astype(int)
meas_piv_selection.to_csv("output/waterhoogtes_selectie.csv", sep=",", encoding="utf-8")

## rhine filtering

In [ ]:
# browse measurement locations
locations.plot()

In [ ]:
rivierlijnen = gpd.read_file("data/rivierlijnen.zip")
rivierlijnen.plot()

In [ ]:
rivierlijnen

In [ ]:
rijnlijn = rivierlijnen.to_crs(epsg=28992).iloc[0].geometry
rijnlijn  # coords start at river head (low Y)

In [ ]:
from shapely import Point, MultiPoint

rijnlijn_vertices = MultiPoint([Point(xy) for xy in rijnlijn.coords])
rijnlijn_vertices

In [ ]:
rijnlijn_locations = locations[locations.distance(rijnlijn) < 1000]
rijnlijn_locations

In [ ]:
from shapely.ops import nearest_points, snap

some_point = rijnlijn_locations.iloc[0].geometry
print("measurement location", some_point)
vertex = nearest_points(some_point, rijnlijn_vertices)[1]
print("vertex using nearest points", vertex)
# list(rijnlijn_vertices.geoms)
index = list(rijnlijn_vertices.geoms).index(vertex)
print(index)


In [ ]:
# construct a dict of locatie_code to geometry index
location_index = dict()
for locatie_code, row in rijnlijn_locations.iterrows():
    vertex = nearest_points(row.geometry, rijnlijn_vertices)[1]
    idx = list(rijnlijn_vertices.geoms).index(vertex)
    location_index[locatie_code] = idx

print(location_index)

In [ ]:
meas_at_time

In [ ]:
{r.LOCATIE_CODE: r.NUMERIEKEWAARDE for _, r in meas_at_time.iterrows()}

In [ ]:
def spatial_interpolate(line_vertices, location_index, measurements, the_time) -> pd.Series:
    """Spatial interpolation of measurement data for a single time point."""
    meas_at_time = measurements[measurements["DATUMTIJD"] == the_time]

    reversed_location_index = {v: k for k, v in location_index.items()}
    points = [reversed_location_index.get(i, None) for i in range(len(line_vertices.geoms))]
    values = [
        np.nan if p is None else int(meas_at_time[meas_at_time["LOCATIE_CODE"] == p]["NUMERIEKEWAARDE"].iloc[0])
        for p in points
    ]
    series = pd.Series(values)
    series = series.interpolate()
    series.name = the_time
    return series

the_time = meas_long.iloc[0]["DATUMTIJD"]
result = spatial_interpolate(rijnlijn_vertices, location_index, meas_long, the_time)
result


In [ ]:
meas_long["DATUMTIJD"].unique()

In [ ]:
all_data = pd.concat([spatial_interpolate(rijnlijn_vertices, location_index, meas_long, the_time) for the_time in meas_long["DATUMTIJD"].unique()], axis=1)
all_data.head()


In [ ]:
all_data.round(3).to_csv("output/ijssel_data.csv", index=False)

In [ ]:
# output ijssel locations
ijssel_locations = gpd.GeoDataFrame(geometry=list(rijnlijn_vertices.geoms))

ijssel_locations["SCALED_X"] = (ijssel_locations.geometry.x - 155_000) / (325_000 / 2)
ijssel_locations["SCALED_Y"] = (ijssel_locations.geometry.y - 463_000) / (325_000 / 2)
ijssel_locations = ijssel_locations.round(5)

sr = [1 for _ in range(len(ijssel_locations))]
sr[0] = 0
sr[-1] = 0
ijssel_locations["SR"] = sr

ijssel_locations.drop(columns="geometry").to_csv("output/ijssel_locations.csv", index=False)

## normalizing the data

In [ ]:
import pandas as pd
meas_piv = pd.read_csv("../scripts/water/output/rws_noordwaard_pivoted.csv")
meas_piv.set_index("LOCATIE_CODE", inplace=True)
meas_piv.head()

In [ ]:
meas_long = meas_piv.reset_index().melt(id_vars="LOCATIE_CODE", var_name="DATUMTIJD", value_name="WAARDE")
meas_long.sort_values(by=["LOCATIE_CODE", "DATUMTIJD"], inplace=True)
meas_long.reset_index(drop=True, inplace=True)
meas_long["DATUMTIJD"] = pd.to_datetime(meas_long["DATUMTIJD"])
meas_long.head()

In [ ]:
# resample the time index to 10 minute intervals
ms = meas_piv.T
ms.index = pd.to_datetime(ms.index)
resampled = ms.resample("10min").mean().astype(int)
resampled.head()

In [ ]:
# try normalizing the measurement data to min and max of first x days
NORMALIZE_NUMBER_OF_DAYS = 2

meas = resampled
meas_first_days = meas[0:NORMALIZE_NUMBER_OF_DAYS*24*6]
mx = meas_first_days.max()
mn = meas_first_days.min()

meas_norm = (meas-mn*0.9)/(mx-mn)
meas_norm[meas_norm < 0] = 0  # avoid negative values, for touch
meas_norm

In [ ]:
meas_norm_long = meas_norm.reset_index().melt(id_vars="index", value_name="WAARDE_GESCHAALD")
meas_norm_long.rename(columns={"index": "DATUMTIJD"}, inplace=True)
meas_norm_long

In [ ]:
import numpy as np
(meas_norm_long["WAARDE_GESCHAALD"] == np.nan).sum()

In [ ]:
merged = pd.merge(meas_long, meas_norm_long)
merged.sort_values(by=["LOCATIE_CODE", "DATUMTIJD"], inplace=True)
merged.head()

In [ ]:
graph_code = "MAASEK"

In [ ]:
merged[merged["LOCATIE_CODE"] == graph_code]["WAARDE"].plot()

In [ ]:
merged[merged["LOCATIE_CODE"] == graph_code]["WAARDE_GESCHAALD"].plot()

In [ ]:
minute_diffs = pd.to_datetime(merged["DATUMTIJD"].unique()).diff() / pd.Timedelta(minutes=1)
pd.Series(minute_diffs).plot()

In [ ]:
# meas_long

merged.to_csv("output/noordwaard_normalized_long.csv", sep=",", encoding="utf-8", index=False)
# meas_norm_long.to_csv("output/noordwaard_normalized_long.csv", sep=",", encoding="utf-8", index=False)


## old stuff

In [ ]:
nederrijn = rijnlijn[rijnlijn.layer == "nederrijn"].to_crs(epsg=28992)
nederrijn.plot()

In [ ]:
nederrijn_geometry = nederrijn.iloc[0].geometry
nederrijn_geometry  # type: LineString

In [ ]:
locations.distance(nederrijn_geometry)  # shortest distance of every location to nederrijn linestring

In [ ]:
nederrijn_locations = locations[locations.distance(nederrijn_geometry) < 1000]
nederrijn_locations.plot()

In [ ]:
nederrijn_locations["RD_X"], nederrijn_locations["RD_Y"] = nederrijn_locations.geometry.xy

In [ ]:
# plot original nederrijn geometry with vertices
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.plot(*zip(*nederrijn_geometry.coords), 'o-')
plt.show()

In [ ]:
# segmentize nederrijn (denser vertices)
nederrijn_dense = nederrijn_geometry.segmentize(max_segment_length=1000)

In [ ]:
# plot densified nederrijn
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.plot(*zip(*nederrijn_dense.coords), 'o-')
plt.show()

In [ ]:
# for every measurement location, find the nearest point on the dense nederrijn geom
import shapely
nederrijn_dense_points = list(nederrijn_dense.coords)  # list of tuples

new_points = []

for measured_geom in nederrijn_locations.geometry:
    distances = [shapely.distance(measured_geom, shapely.Point(ndp)) for ndp in nederrijn_dense_points]
    min_distance = min(distances)
    index = distances.index(min_distance)
    new_point = nederrijn_dense_points[index]
    new_points.append(shapely.Point(new_point))

nederrijn_locations_mapped = nederrijn_locations.copy()
nederrijn_locations_mapped.geometry = new_points

for i in range(len(nederrijn_locations)):
    print(nederrijn_locations.iloc[i].geometry, "to", nederrijn_locations_mapped.iloc[i].geometry)

## Random analysis

In [ ]:
import pandas as pd
noordwaard = pd.read_csv("../scripts/water/output/rws_noordwaard_pivoted.csv")
noordwaard.head()

In [ ]:
werkendam = noordwaard[noordwaard["LOCATIE_CODE"] == "WERKDBTN"].drop(columns="LOCATIE_CODE").squeeze()
werkendam

In [ ]:
werkendam.plot()

In [ ]:
import pandas as pd
data = pd.read_csv("output/noordwaard_normalized_long.csv")
data.head()

In [ ]:
data["LOCATIE_CODE"].unique()

In [ ]:
data[data["LOCATIE_CODE"] == "MAASEK"]["WAARDE_GESCHAALD"].plot()

## finding the correct time for Lobith laagwater

In [ ]:
import pandas as pd
lobith = pd.read_csv("data/20250401_077.csv",  sep=";", encoding="iso-8859-1")
lobith["NUMERIEKEWAARDE"].plot()  # lowest value was 632

In [ ]:
lobith[lobith["NUMERIEKEWAARDE"] == 632]["WAARNEMINGDATUM"]

In [ ]:
# lange dataset van 1 station om te bepalen wat een goede 'normaalperiode' is
# conclusie: half maart 2024 (15 t/m 22 maart)
import pandas as pd
import numpy as np

data = pd.read_csv("data/tijdelijk_analyse/20250402_046.csv", sep=";", encoding="iso-8859-1")

data.replace(999_999_999.0, np.nan, inplace=True)  # RWS marks bad values with high 9s

# add proper datetime
data["DATUMTIJD"] = pd.to_datetime(
    data["WAARNEMINGDATUM"] + " " + data["WAARNEMINGTIJD (MET/CET)"],
    format="%d-%m-%Y %H:%M:%S",
)

data.index = data["DATUMTIJD"]

data["NUMERIEKEWAARDE"].plot()

## Bepalen van normaalstand per meetpunt (laag/hoog grens) obv 1 week aan data

In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("data/tijdelijk_analyse/20250402_048.csv", sep=";", encoding="iso-8859-1")

# add proper datetime
data["DATUMTIJD"] = pd.to_datetime(
    data["WAARNEMINGDATUM"] + " " + data["WAARNEMINGTIJD (MET/CET)"],
    format="%d-%m-%Y %H:%M:%S",
)

data["DATUMTIJD"].unique()

In [ ]:
    data.replace(999_999_999.0, np.nan, inplace=True)  # RWS marks bad values with high 9s
    data.dropna(axis="columns", how="all", inplace=True)

In [ ]:
pivoted = data.pivot_table(index="DATUMTIJD", values="NUMERIEKEWAARDE", columns="LOCATIE_CODE")
pivoted.max()

In [ ]:
    # remove outliers by looking at mean of absolute diff
    abs_diff = pivoted.diff().abs()
    outliers = abs_diff >= 5 * abs_diff.mean()
    pivoted[outliers] = np.nan

In [ ]:
pivoted.min()

In [ ]:
pivoted.max()

In [ ]:
min_values = pivoted.min()
min_values.name = "GRENS_LAAG"

max_values = pivoted.max()
max_values.name = "GRENS_HOOG"

pd.merge(min_values, max_values, left_index=True, right_index=True).to_csv("bla.csv", sep=",", encoding="utf-8")